# Study case 

## Vehicle collision reports in the Monterrey Metropolitan area

### Data Set overview

In [1]:
# libraries
import pandas as pd
import plotly.express as px
import plotly.figure_factory as ff
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('data.csv')
df.head(5)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning:

Columns (9,12,13,16,21,28,32,33,34,35,36,37,40,41,42,43,44,45,46,49) have mixed types.Specify dtype option on import or set low_memory=False.



,Column,Calle,Causa,Ciudad,Cruce,Fecha,Hora,ID,Latitud,Lesionados,Longitud,Mes,Muertos,No. De Infracción,Número total de lesionados,Número total de muertos,Por alcohol,Sentido,Situación climática,Situación del Pavimento,Tipo de incidente,Tipo de involucrado,Tipo de vía,V1_Arresto,V1_Dictamen,V1_Edad,V1_Estatus Licencia,V1_Género,V1_No. De Fallecidos,V1_No. De Lesionados,V1_Tipo de Vehículo,V1_Uso de Cinturón,V2_Arresto,V2_Dictamen,V2_Edad,V2_Estatus Licencia,V2_Género,V2_No. De Fallecidos,V2_No. De Lesionados,V2_Tipo de Vehículo,V2_Uso de Cinturón,V3_Arresto,V3_Dictamen,V3_Edad,V3_Estatus Licencia,V3_Género,V3_No. De Fallecidos,V3_No. De Lesionados,V3_Tipo de Vehículo,V3_Uso de Cinturón
0,0,CAMINO REAL,NO RESPETAR SEÑALAMIENTOS,Monterrey,SANTOS PALOMO,2016-05-01 00:00:00,1899-12-30 00:00:00,MTY1,25.77443,No,-100.371506,Mayo,No,NaN,NaN,NaN,NaN,NORTE A SUR,Normal,Normal,Choque,Conductor,Vía secundaria,NaN,NaN,SD,SIN,M,NaN,NaN,Automóvil,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Automóvil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,BERNARDO REYES,Otros,Monterrey,FIDEL VELAZQUEZ,2016-05-01 00:00:00,1899-12-30 00:00:00,MTY10,25.724161,No,-100.327445,Mayo,No,NaN,NaN,NaN,NaN,Oriente a Poniente,Normal,Normal,Choque,Conductor,Vía primaria,NaN,NaN,57,SIN,M,NaN,NaN,Automóvil,Sí,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Automóvil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,BERNARDO REYES,Otros,Monterrey,REFORMA,2016-05-01 00:00:00,1899-12-30 00:00:00,MTY11,25.685874,No,-100.322931,Mayo,No,NaN,NaN,NaN,NaN,NORTE A SUR,Normal,Normal,Choque,Conductor,Vía primaria,NaN,NaN,34,SIN,M,NaN,NaN,Automóvil,Sí,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Automóvil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,LINCOLN,INVADIR CARRIL,Monterrey,CUMBRES DEL SOL,2016-05-01 00:00:00,1899-12-30 00:00:00,MTY12,25.797401,No,-100.184189,Mayo,No,NaN,NaN,NaN,NaN,Oriente a Poniente,Normal,Normal,Lateral,Conductor,Vía primaria,NaN,NaN,27,Vigente,M,NaN,NaN,Camión de Carga,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Automóvil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,MADERO,NO GUARDAR DISTANCIA,Monterrey,PABLO A DE LA GARZA,2016-05-01 00:00:00,1899-12-30 00:00:00,MTY13,25.682219,No,-100.287349,Mayo,No,NaN,NaN,NaN,NaN,NORTE A SUR,Normal,Normal,Alcance,Conductor,Vía primaria,NaN,NaN,39,SIN,M,NaN,NaN,Automóvil,Sí,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Automóvil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df.tail(5)

,Column,Calle,Causa,Ciudad,Cruce,Fecha,Hora,ID,Latitud,Lesionados,Longitud,Mes,Muertos,No. De Infracción,Número total de lesionados,Número total de muertos,Por alcohol,Sentido,Situación climática,Situación del Pavimento,Tipo de incidente,Tipo de involucrado,Tipo de vía,V1_Arresto,V1_Dictamen,V1_Edad,V1_Estatus Licencia,V1_Género,V1_No. De Fallecidos,V1_No. De Lesionados,V1_Tipo de Vehículo,V1_Uso de Cinturón,V2_Arresto,V2_Dictamen,V2_Edad,V2_Estatus Licencia,V2_Género,V2_No. De Fallecidos,V2_No. De Lesionados,V2_Tipo de Vehículo,V2_Uso de Cinturón,V3_Arresto,V3_Dictamen,V3_Edad,V3_Estatus Licencia,V3_Género,V3_No. De Fallecidos,V3_No. De Lesionados,V3_Tipo de Vehículo,V3_Uso de Cinturón
55008,25251,5 DE MAYO,NaN,San Pedro,DEGOLLADO,2017-04-30 00:00:00,09:12:00 a.m.,SP2212,NaN,NaN,NaN,ABRIL,NaN,NaN,NaN,NaN,NaN,Poniente a Oriente,NaN,NaN,SD,NaN,NaN,No,NaN,69,Vigente,M,0,0.0,Automóvil Privado,Sí,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
55009,25252,MANUEL SANTOS,NaN,San Pedro,HUMBERTO LOBO,2017-04-30 00:00:00,02:17:00 p.m.,SP2213,NaN,NaN,NaN,ABRIL,NaN,NaN,NaN,NaN,NaN,NORTE A SUR,NaN,NaN,SD,NaN,NaN,No,NaN,70,Vigente,M,0,0.0,Automóvil Privado,Sí,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
55010,25253,CALZADA DEL VALLE,NaN,San Pedro,CALZADA SAN PEDRO,2017-04-30 00:00:00,03:41:00 p.m.,SP2214,NaN,NaN,NaN,ABRIL,NaN,NaN,NaN,NaN,NaN,Oriente a Poniente,NaN,NaN,SD,NaN,NaN,No,NaN,19,Vigente,M,0,0.0,Motocicleta,Sí,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
55011,25254,JOSE VASCONCELOS,NaN,San Pedro,CALZADA SAN PEDRO,2017-04-30 00:00:00,07:41:00 p.m.,SP2215,NaN,NaN,NaN,ABRIL,NaN,NaN,NaN,NaN,NaN,Poniente a Oriente,NaN,NaN,SD,NaN,NaN,No,NaN,15,Vencida,M,0,0.0,Automóvil Privado,Sí,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
55012,25255,JOSE VASCONCELOS,NaN,San Pedro,J. M. MONTEMAYOR,2017-04-30 00:00:00,09:03:00 p.m.,SP2216,NaN,NaN,NaN,ABRIL,NaN,NaN,NaN,NaN,NaN,Poniente a Oriente,NaN,NaN,SD,NaN,NaN,No,NaN,45,Vigente,M,0,0.0,Automóvil Privado,Sí,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55013 entries, 0 to 55012
Data columns (total 50 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Column                       55013 non-null  int64  
 1   Calle                        54998 non-null  object 
 2   Causa                        48986 non-null  object 
 3   Ciudad                       55009 non-null  object 
 4   Cruce                        53444 non-null  object 
 5   Fecha                        55010 non-null  object 
 6   Hora                         51656 non-null  object 
 7   ID                           55009 non-null  object 
 8   Latitud                      25551 non-null  object 
 9   Lesionados                   27935 non-null  object 
 10  Longitud                     34515 non-null  object 
 11  Mes                          54908 non-null  object 
 12  Muertos                      22379 non-null  object 
 13  No. De Infracció

In [5]:
# Looking for nulls

df.isnull().sum()

Column                             0
Calle                             15
Causa                           6027
Ciudad                             4
Cruce                           1569
Fecha                              3
Hora                            3357
ID                                 4
Latitud                        29462
Lesionados                     27078
Longitud                       20498
Mes                              105
Muertos                        32634
No. De Infracción              52201
Número  total de lesionados    27006
Número total de muertos        33522
Por alcohol                    35742
Sentido                         2423
Situación climática             7091
Situación del Pavimento         7024
Tipo de incidente                  0
Tipo de involucrado            26459
Tipo de vía                    12731
V1_Arresto                     15180
V1_Dictamen                    43205
V1_Edad                         3646
V1_Estatus Licencia             6218
V

### 1. Which roads "appear to be" the top 10 most dangerous roads in the Monterrey Metropolitan area?

In [6]:
# Chosing only a few columns for this question

df_1 = df[['Calle','Causa','Ciudad','Cruce','Latitud','Longitud','Lesionados','Muertos']]
df_1.dropna(subset=['Calle'], thresh=1, inplace=True)
df_1.head()

,Calle,Causa,Ciudad,Cruce,Latitud,Longitud,Lesionados,Muertos
0,CAMINO REAL,NO RESPETAR SEÑALAMIENTOS,Monterrey,SANTOS PALOMO,25.77443,-100.371506,No,No
1,BERNARDO REYES,Otros,Monterrey,FIDEL VELAZQUEZ,25.724161,-100.327445,No,No
2,BERNARDO REYES,Otros,Monterrey,REFORMA,25.685874,-100.322931,No,No
3,LINCOLN,INVADIR CARRIL,Monterrey,CUMBRES DEL SOL,25.797401,-100.184189,No,No
4,MADERO,NO GUARDAR DISTANCIA,Monterrey,PABLO A DE LA GARZA,25.682219,-100.287349,No,No


In [7]:
df_1.isnull().sum()

Calle             0
Causa          6020
Ciudad            0
Cruce          1558
Latitud       29452
Longitud      20489
Lesionados    27066
Muertos       32622
dtype: int64

In [8]:
# Lesionados values = {'No':24058, 'Sí':865, '0':1984, 'SD':945, '1':62, '2':10, '3':6, '4':1, '5':1, nan:...}
# df.Muertos.unique()
df_1['Lesionados'].value_counts()

No    24058
0      1984
SD      945
Sí      865
1        62
2        10
3         6
4         1
5         1
Name: Lesionados, dtype: int64

In [9]:
# grouped by street and city because it's possible to have same streen name between cities even in the same city

streets = (df_1.groupby(['Calle', 'Ciudad']).size() 
   .sort_values(ascending=False) 
   .reset_index(name='Count'))

fig =  ff.create_table(streets.head(10))
fig.show()

In [10]:
# lets watch heatmap based on lat and long
import re

#df_lat_long = df_1[['Latitud','Longitud']][(len(str(df_1['Latitud'])) > 5) & (len(str(df_1['Longitud'])) > 8)]
#df_lat_long.Longitud = df_1.Longitud.apply(lambda x: 'nan' if '.' not in str(x) else re.findall('^(-?\d+(\.\d+)?)', str(x)))
#df_lat_long.Latitud = df_1.Latitud.apply(lambda x: re.search(r'^(-?\d+(\.\d+)?)', str(x)) if (len(str(x)) > 5 and '.' in str(x)) else 'nan')
#df_lat_long.Longitud = df_1.Longitud.apply(lambda x: re.findall(r'\w*(\-?\d+(\.\d+)?)$', str(x)) if (len(str(x)) > 8 and '.' in str(x)) else 'nan') 
df_lat_long = df_1[['Latitud','Longitud']]
df_lat_long.Latitud = df_lat_long.Latitud.apply(lambda x: str(x).replace(',','').replace('+','') if (re.match(r'^(-?\d+(\.\d+)?)', str(x)) and '.' in str(x)) else 'nan')
df_lat_long.Longitud = df_lat_long.Longitud.apply(lambda x: x if (re.match(r'\w*(\-?\d+(\.\d+)?)$', str(x)) and '.' in str(x)) else 'nan')
df_lat_long = df_lat_long[['Latitud','Longitud']][ (df_lat_long['Latitud'] != 'nan') & (df_lat_long['Longitud'] != 'nan')]

df_lat_long['Latitud'] = df_lat_long['Latitud'].astype(float)
df_lat_long['Longitud'] = df_lat_long['Longitud'].astype(float)
df_lat_long.head()

,Latitud,Longitud
0,25.774430,-100.371506
1,25.724161,-100.327445
2,25.685874,-100.322931
3,25.797401,-100.184189
4,25.682219,-100.287349


In [11]:
import folium
from folium import plugins
from folium.plugins import HeatMap

In [13]:
mapa3=folium.Map(location=[25.724161,-100.327445],zoom_start=12)

data=df_lat_long[['Latitud','Longitud']].values

mapa3.add_child(plugins.HeatMap(data, radius=15))

mapa3

### 4. Why do you think we use the phrase "appear to be" in question 1? What would you need to be able to speak with more confidence about the answer to that question?

In [12]:
# Top 10 by deaths based on 22379 rows
# Muertos values = {'No':19942, 'Sí':362, '0':2020, 'SD':43, '1':6, 'HUYO':3, 'SE RETIRO':2, nan:...}
df_death = df[['Calle','Ciudad','Muertos']]
df_death['Muertos'] = df_death['Muertos'].fillna('0')
df_death = df_death[(df_death['Muertos'] != 'No') & (df_death['Muertos'] != '0') ] 
df_death['Muertos'].value_counts()
#df_death['Muertos'].unique()

Sí           362
SD            43
1              6
HUYO           3
SE RETIRO      2
Name: Muertos, dtype: int64

In [13]:
top_deaths = (df_death.groupby(['Calle', 'Ciudad']).size() 
   .sort_values(ascending=False) 
   .reset_index(name='Count'))

top_deaths.head(10)

,Calle,Ciudad,Count
0,AZTLAN,Monterrey,15
1,MADERO,Monterrey,11
2,COLON,Monterrey,9
3,JUAREZ,Monterrey,9
4,RUIZ CORTINEZ,Monterrey,7
5,LUIS D COLOSIO,Monterrey,7
6,LEONES,Monterrey,7
7,CAMINO REAL,Monterrey,6
8,PINO SUAREZ,Monterrey,6
9,MORONES PRIETO,Monterrey,5


In [14]:
# Top 10 by people injuried based on 27935 rows

df_injuries = df[['Calle','Ciudad','Lesionados']]
df_injuries['Muertos'] = df_injuries['Lesionados'].fillna('0')
df_injuries = df_injuries[(df_injuries['Muertos'] != 'No') & (df_injuries['Muertos'] != '0') ] 
df_injuries['Lesionados'].value_counts()
#df_injuries['Lesionados'].unique()

SD    946
Sí    865
1      62
2      10
3       6
5       1
4       1
Name: Lesionados, dtype: int64

In [15]:
top_injuries = (df_injuries.groupby(['Calle', 'Ciudad']).size() 
   .sort_values(ascending=False) 
   .reset_index(name='Count'))

top_injuries.head(10)

,Calle,Ciudad,Count
0,HUMBERTO RAMOS LOZANO,Apodaca,46
1,MIGUEL ALEMAN,Apodaca,36
2,AV. LINCOLN,García,36
3,CONCORDIA,Apodaca,30
4,BLVD CARLOS SALINAS DE GORTARI,Apodaca,30
5,CARR MIGUEL ALEMAN,Apodaca,20
6,BLVD HEBERTO CASTILLO,García,18
7,AZTLAN,Monterrey,17
8,JOSE VASCONCELOS,San Pedro,17
9,ISIDORO SEPULVEDA,Apodaca,17


### 2. What's the age group with the overall highest number of reported collisions? You can consider any age groupings for the purposes of this question. Explain this in a graphical way.

In [61]:
df_ages = df[['V1_Edad','V2_Edad','V3_Edad']]
df_ages['V1_Edad'] = df_ages['V1_Edad'].apply(lambda x: x if re.match(r'(\d{2}$)',str(x)) else '0')
df_ages['V2_Edad'] = df_ages['V2_Edad'].apply(lambda x: x if re.match(r'(\d{2}$)',str(x)) else '0')
df_ages['V3_Edad'] = df_ages['V3_Edad'].apply(lambda x: x if re.match(r'(\d{2}$)',str(x)) else '0')

In [62]:
df_ages.head()

,V1_Edad,V2_Edad,V3_Edad
0,0,0,0
1,57,0,0
2,34,0,0
3,27,0,0
4,39,0,0


In [63]:
# V1_Edad graph age

age_v1 = df_ages[df_ages['V1_Edad'] != '0']
age_v1 = (age_v1.groupby(['V1_Edad']).size()  
   .reset_index(name='Count'))

fig = px.bar(age_v1, y='Count', x='V1_Edad', text='Count')
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.show()


In [64]:
# V1_Edad graph age

age_v2 = df_ages[df_ages['V2_Edad'] != '0']
age_v2 = (age_v2.groupby(['V2_Edad']).size()  
   .reset_index(name='Count'))

fig = px.bar(age_v2, y='Count', x='V2_Edad', text='Count')
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.show()

In [65]:
age_v3 = df_ages[df_ages['V3_Edad'] != '0']
age_v3 = (age_v3.groupby(['V3_Edad']).size()  
   .reset_index(name='Count'))

fig = px.bar(age_v3, y='Count', x='V3_Edad', text='Count')
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.show()

In [85]:
age_v1.head()

,V1_Edad,Count
0,11,1
1,12,3
2,13,2
3,14,6
4,15,40


In [89]:
from collections import Counter

#age_v1.rename({'V1_Edad':'Edad'}, axis=1, inplace=True)
#age_v2.rename({'V2_Edad':'Edad'}, axis=1, inplace=True)
#age_v3.rename({'V3_Edad':'Edad'}, axis=1, inplace=True)
v1 = pd.Series(age_v1.Count.values, age_v1.V1_Edad.values).to_dict()
v2 = pd.Series(age_v2.Count.values, age_v2.V2_Edad.values).to_dict()
v3 = pd.Series(age_v3.Count.values, age_v3.V3_Edad.values).to_dict()

full_ages_dict = dict(Counter(v1) + Counter(v2)+ Counter(v3))

total_ages = pd.DataFrame(full_ages_dict.items(), columns=['Edad', 'Casos'])
 
total_ages.head(10)
#print(full_ages_dict)

,Edad,Casos
0,11,3
1,12,6
2,13,4
3,14,13
4,15,49
5,16,189
6,17,340
7,18,780
8,19,979
9,20,1310


In [95]:
print(full_ages_dict)

{'11': 3, '12': 6, '13': 4, '14': 13, '15': 49, '16': 189, '17': 340, '18': 780, '19': 979, '20': 1310, '21': 1391, '22': 1551, '23': 1795, '24': 1874, '25': 1936, '26': 1986, '27': 1974, '28': 1972, '29': 1790, '30': 2051, '31': 1786, '32': 1893, '33': 1741, '34': 1937, '35': 1664, '36': 1760, '37': 1639, '38': 1497, '39': 1485, '40': 1524, '41': 1394, '42': 1605, '43': 1436, '44': 1224, '45': 1402, '46': 1182, '47': 1109, '48': 1042, '49': 928, '50': 976, '51': 790, '52': 812, '53': 826, '54': 716, '55': 669, '56': 700, '57': 567, '58': 576, '59': 490, '60': 472, '61': 376, '62': 385, '63': 306, '64': 301, '65': 284, '66': 250, '67': 215, '68': 188, '69': 165, '70': 179, '71': 133, '72': 109, '73': 100, '74': 107, '75': 90, '76': 61, '77': 52, '78': 50, '79': 41, '80': 37, '81': 29, '82': 24, '83': 20, '84': 16, '85': 17, '86': 7, '87': 5, '88': 7, '89': 1, '90': 1, '94': 2, '96': 1, '10': 5, '92': 2}


In [93]:
fig = px.bar(total_ages, x='Edad', y='Casos',
             hover_data=['Edad', 'Casos'], color='Edad',
             labels={'pop':'colisiones'}, height=400)
fig.show()

In [114]:
def grouping_by_5(x):
  print(x)
  by_5 = {}
  
  for i in range(15, 100, 5):
    k = 0
    a = i - 5
    for key, val in x.items():
      if int(key) > a and int(key) <= i:
        k += val
    l = ('{}-{}'.format(a,i))
    #print(l)
    by_5.update({l:k})
    a += i
  return by_5

sets_5 = grouping_by_5(full_ages_dict)
print(sets_5)

{'11': 3, '12': 6, '13': 4, '14': 13, '15': 49, '16': 189, '17': 340, '18': 780, '19': 979, '20': 1310, '21': 1391, '22': 1551, '23': 1795, '24': 1874, '25': 1936, '26': 1986, '27': 1974, '28': 1972, '29': 1790, '30': 2051, '31': 1786, '32': 1893, '33': 1741, '34': 1937, '35': 1664, '36': 1760, '37': 1639, '38': 1497, '39': 1485, '40': 1524, '41': 1394, '42': 1605, '43': 1436, '44': 1224, '45': 1402, '46': 1182, '47': 1109, '48': 1042, '49': 928, '50': 976, '51': 790, '52': 812, '53': 826, '54': 716, '55': 669, '56': 700, '57': 567, '58': 576, '59': 490, '60': 472, '61': 376, '62': 385, '63': 306, '64': 301, '65': 284, '66': 250, '67': 215, '68': 188, '69': 165, '70': 179, '71': 133, '72': 109, '73': 100, '74': 107, '75': 90, '76': 61, '77': 52, '78': 50, '79': 41, '80': 37, '81': 29, '82': 24, '83': 20, '84': 16, '85': 17, '86': 7, '87': 5, '88': 7, '89': 1, '90': 1, '94': 2, '96': 1, '10': 5, '92': 2}
{'10-15': 75, '15-20': 3598, '20-25': 8547, '25-30': 9773, '30-35': 9021, '35-40': 

In [117]:
groups = pd.DataFrame(sets_5.items(), columns=['Rango', 'Casos'])
fig = px.pie(groups, values='Casos', names='Rango', title='colisiones por rango de edad')
fig.show()

### 3. If you were to see a collision in the "Garza Sada" road, What would be the odds for that person to have any type of positive alcohol ingestion opinion?

In [29]:
df_3 = df[['Calle','Lesionados','Muertos','Por alcohol']][df['Calle'] == 'GARZA SADA']
df_3['Lesionados'] = df_3.Lesionados.apply(lambda x: 1 if re.match(r'(Sí$)',str(x)) else 0)
df_3['Muertos'] = df_3.Muertos.apply(lambda x: 1 if re.match(r'(Sí$)',str(x)) else 0)
df_3['Por alcohol'] = df_3['Por alcohol'].apply(lambda x: 1 if re.match(r'(Sí$)',str(x)) else 0)
df_3['Total_colisiones'] = 1
df_3.head()

,Calle,Lesionados,Muertos,Por alcohol,Total_colisiones
12,GARZA SADA,0,0,0,1
14,GARZA SADA,0,0,0,1
17,GARZA SADA,0,0,0,1
51,GARZA SADA,0,0,0,1
57,GARZA SADA,0,0,0,1


In [31]:
#df_3['Por alcohol'].unique()
df_alcohol = df_3.groupby(['Calle']).sum()
df_alcohol

,Lesionados,Muertos,Por alcohol,Total_colisiones
Calle,,,,
GARZA SADA,13,4,82,1215


In [45]:
a = (df_alcohol.iloc[0]['Por alcohol'] / df_alcohol.iloc[0]['Total_colisiones']) * 100

print('Of the collisions in the "GARZA SADA" road, {} % of the people would think that driving while drunk is a bad fact, \nthe rest could remain in a neutral or pisitive opinion.'.format(a.round()))

Of the collisions in the "GARZA SADA" road, 7.0 % of the people would think that driving while drunk is a bad fact, 
the rest could remain in a neutral or pisitive opinion.


### 5. What modelling project would you push to develop from the conclusions in this dataset?

1.- Time series.

2.- Try clustering models to unveil hidden patterns